In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("rshaojimmy/DGM4", split='validation')
train_dataset

In [ ]:
train_set = []
real = 0
fake = 0

for el in train_dataset:
    if (el['image'].split('/')[2] == 'washington_post') or (el['image'].split('/')[2] == 'simswap'):
        train_set.append({
            'text': el['text'],
            'image': el['image']
        })
        if el['image'].split('/')[1] == 'origin':
            real += 1
        else:
            fake += 1
    
len(train_set)

In [6]:
from TT_BLIP.batch_extractor_dgm import DatasetLoader
from TT_BLIP.tt_blip_layers import TT_BLIP_Model
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import torch



ds_loader = DatasetLoader(batch_size=1)
train_dl, val_dl = ds_loader.get_dataloaders()


In [2]:
ckpt = torch.load("./models/blip_unlocked.ckpt", map_location='cpu')

/tmp/ipykernel_112500/3177015801.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("./models/blip_unlocked.ckpt", map_location='cpu')


In [3]:
model = TT_BLIP_Model(
        empty_img=ds_loader.dp.empty_pixel_values, 
        empty_txt=ds_loader.dp.empty_input_ids,
        empty_attn_mask=ds_loader.dp.empty_attn_mask, 
        embed_dim=1024, 
        num_heads=16,
        trainable=0
    )


In [4]:
model.load_state_dict(ckpt['state_dict'])
model.eval()

TT_BLIP_Model(
  (feature_extraction_layer): FeatureExtractionLayer(
    (vit): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): ViTPatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0-11): 12 x ViTLayer(
            (attention): ViTSdpaAttention(
              (attention): ViTSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
    

In [ ]:
from tqdm.auto import tqdm

cm = torch.zeros((2, 2))
for b in tqdm(val_dl):
    x, y = b
    with torch.no_grad():
        pred = model(x).cpu()
    
    i = (pred > 0.5).int()
    j = y.int()

    for idx in range(ds_loader.batch_size):
        cm[i[idx].item(), j[idx].item()] += 1

cm



  0%|          | 0/3655 [00:00<?, ?it/s]